In [1]:
import pandas as pd
x_train=pd.read_csv('training_input_qFI2jug.csv', index_col="Horodate_UTC")
x_train.index = pd.DatetimeIndex(x_train.index)

x_train.head()




,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,11876221,0.0,608873.0,9.6,13.5,302936853.0,19660.0,NaN,9072106039,436423300
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,11719006,0.0,608873.0,9.6,13.5,368725682.0,17004.0,NaN,9031111381,407924660
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,11742773,0.0,608873.0,9.6,13.5,448572685.0,24299.0,NaN,8949500729,398239990
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,11719010,0.0,608873.0,9.5,13.5,550243295.0,23308.0,NaN,8882842849,380571680
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,11731807,0.0,608873.0,9.5,13.5,644783673.0,21963.0,NaN,8829957877,367153030


I think, We need to use the pseudo radiation column in the time series analysis.
And we can drop this column when we will train the model.


In [2]:
# Define helper function
# def fill_missing(grp):
#     res = grp.set_index('date')\
#     .interpolate(method='linear',limit=5)\
#     .ffill()\
#     .bfill()
#     del res['Pseudo rayonnement']
#     return res# Group by country name and fill missing
# 
# df = x_train
# 
# df['dt_Horodate_UTC'] = pd.to_datetime(df['Horodate_UTC'], infer_datetime_format=True)
# 
# df['date'] = df['dt_Horodate_UTC'].map(lambda x: x.strftime('%d/%m %H:%S'))
# 
# df = df.groupby(['Pseudo rayonnement']).apply(
#     lambda grp: fill_missing(grp)
# )
# 
# df = df.reset_index()

df = x_train

liste_dates = df.index.tolist()
for dt in liste_dates :
    df.at[dt, 'day'] = dt.strftime('%d/%m %H:%M')
    

df_month = df.sort_values(by='day', ascending=True)
df_month.head(20)
df_month = df_month.interpolate(str="linear", axis=0).ffill().bfill()
df_month = df_month.sort_values(by='IDS', ascending=True)

if 'day' in x_train.columns:
    x_train.drop('day', axis=1, inplace=True)

x_train['Pseudo rayonnement'] = df_month['Pseudo rayonnement']

x_train.head(20)

,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,11876221,0.0,608873.0,9.6,13.5,3.029369e+08,19660.0,69.159021,9072106039,436423300
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,11719006,0.0,608873.0,9.6,13.5,3.687257e+08,17004.0,69.212029,9031111381,407924660
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,11742773,0.0,608873.0,9.6,13.5,4.485727e+08,24299.0,69.318043,8949500729,398239990
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,11719010,0.0,608873.0,9.5,13.5,5.502433e+08,23308.0,69.450561,8882842849,380571680
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,11731807,0.0,608873.0,9.5,13.5,6.447837e+08,21963.0,50.124460,8829957877,367153030
2013-10-14 00:30:00,6,2013-10-13T02:30:00+02:00,10,32989918000,218749000,2237743365,31811786665,7890473020,23921313645,911513262,...,11719010,0.0,608873.0,9.5,13.4,7.440487e+08,23600.0,50.342264,8801986282,345164660
2013-10-14 01:00:00,7,2013-10-13T03:00:00+02:00,10,31555221000,286655000,2114206795,30572424807,7878131750,22694293057,897252387,...,11755572,0.0,608873.0,9.4,13.4,8.640563e+08,20637.0,50.466724,8775384137,326120710
2013-10-14 01:30:00,8,2013-10-13T03:30:00+02:00,10,30358247000,332130000,2012649959,29543363726,7869134200,21674229526,899179610,...,11803104,0.0,608873.0,9.4,13.4,9.622568e+08,19622.0,50.528954,8768313810,313636680
2013-10-14 02:00:00,9,2013-10-13T04:00:00+02:00,10,29634878000,373161000,1951941973,28919999723,7857298140,21062701583,903294176,...,12031617,0.0,608873.0,9.4,13.4,1.032753e+09,17640.0,50.653414,8760592316,303554290


In [3]:
from jours import holydays, summer_winter_time

changeover = summer_winter_time(x_train)

changeover.split('Horodate')


In [4]:
summer_time = changeover.summer_time()
summer_time.head()


,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,11876221,0.0,608873.0,9.6,13.5,302936853.0,19660.0,69.159021,9072106039,436423300
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,11719006,0.0,608873.0,9.6,13.5,368725682.0,17004.0,69.212029,9031111381,407924660
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,11742773,0.0,608873.0,9.6,13.5,448572685.0,24299.0,69.318043,8949500729,398239990
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,11719010,0.0,608873.0,9.5,13.5,550243295.0,23308.0,69.450561,8882842849,380571680
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,11731807,0.0,608873.0,9.5,13.5,644783673.0,21963.0,50.124460,8829957877,367153030


In [5]:
winter_time = changeover.winter_time()
winter_time.head()

,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-28 01:00:00,679,2013-10-27T02:00:00+01:00,10,24459666000,3201714000,1532253405,25313829338,7606637680,17707191658,856172708,...,13672053,0.0,608525.0,16.8,11.5,4.868304e+09,23002.0,99.744214,8462810388,216480690
2013-10-28 01:30:00,680,2013-10-27T02:30:00+01:00,10,23375802000,3293724000,1447608994,24251375532,7579418810,16671956722,895502521,...,13602586,0.0,608525.0,16.8,11.5,4.896575e+09,21907.0,99.676005,8474921331,205722940
2013-10-28 02:00:00,681,2013-10-27T03:00:00+01:00,10,22856999000,3441096000,1407491691,23721592891,7566431260,16155161631,931770173,...,13616142,0.0,608525.0,16.8,11.5,4.986335e+09,21927.0,99.641900,8498201433,200179680
2013-10-28 02:30:00,682,2013-10-27T03:30:00+01:00,10,22481941000,3484826000,1378650442,23371139708,7546684550,15824455158,979268804,...,13658500,0.0,608525.0,16.8,11.5,5.022224e+09,21596.0,99.590743,8525953354,196666010
2013-10-28 03:00:00,683,2013-10-27T04:00:00+01:00,10,22292748000,3501652000,1364153040,23180781322,7539947600,15640833722,1023235452,...,13938063,0.0,608525.0,16.7,11.4,5.022595e+09,21595.0,99.522533,8563183052,195489990


In [6]:
date = holydays()

# date.set_extradaysoff(True)

date.Dataframe(summer_time)

# date.split()

In [7]:
summer_time_workday_df = date.business_days()

summer_time_workday_df.head(20)

,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-14 00:00:00,5,2013-10-13T02:00:00+02:00,10,34882911000,191859000,2403762298,33447730318,7922448360,25525281958,907509517,...,11731807,0.0,608873.0,9.5,13.5,6.447837e+08,2.196300e+04,50.124460,8829957877,367153030
2013-10-14 00:30:00,6,2013-10-13T02:30:00+02:00,10,32989918000,218749000,2237743365,31811786665,7890473020,23921313645,911513262,...,11719010,0.0,608873.0,9.5,13.4,7.440487e+08,2.360000e+04,50.342264,8801986282,345164660
2013-10-14 01:00:00,7,2013-10-13T03:00:00+02:00,10,31555221000,286655000,2114206795,30572424807,7878131750,22694293057,897252387,...,11755572,0.0,608873.0,9.4,13.4,8.640563e+08,2.063700e+04,50.466724,8775384137,326120710
2013-10-14 01:30:00,8,2013-10-13T03:30:00+02:00,10,30358247000,332130000,2012649959,29543363726,7869134200,21674229526,899179610,...,11803104,0.0,608873.0,9.4,13.4,9.622568e+08,1.962200e+04,50.528954,8768313810,313636680
2013-10-14 02:00:00,9,2013-10-13T04:00:00+02:00,10,29634878000,373161000,1951941973,28919999723,7857298140,21062701583,903294176,...,12031617,0.0,608873.0,9.4,13.4,1.032753e+09,1.764000e+04,50.653414,8760592316,303554290
2013-10-14 02:30:00,10,2013-10-13T04:30:00+02:00,10,29220206000,448540000,1917367316,28574876855,7863666930,20711209925,909053910,...,12499612,0.0,608873.0,9.3,13.3,1.138875e+09,2.324700e+04,50.808989,8772720840,295688680
2013-10-14 03:00:00,11,2013-10-13T05:00:00+02:00,10,29279904000,503287000,1922334676,28668367602,7900744470,20767623132,910809517,...,12801248,0.0,608873.0,9.2,13.3,1.229786e+09,2.159000e+04,50.902334,8811553987,293650270
2013-10-14 03:30:00,12,2013-10-13T05:30:00+02:00,10,29339176000,551695000,1927269969,28772583440,7925256630,20847326810,924292549,...,12751889,0.0,608873.0,9.2,13.3,1.334063e+09,1.894200e+04,50.995678,8849549179,298448980
2013-10-14 04:00:00,13,2013-10-13T06:00:00+02:00,10,29923167000,635255000,1976076190,29377866297,7837626840,21540239457,925394967,...,12704357,0.0,608873.0,9.1,13.2,1.493083e+09,2.489300e+04,51.151253,8763021807,303413310


In [8]:
summer_time_dayoff_df = date.public_holiday()

summer_time_dayoff_df.head(20)


,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-13 22:00:00,1,2013-10-13T00:00:00+02:00,10,39784230000,103415000,2849582877,37576289504,8062590100,29513699404,1009515939,...,11876221,0.0,608873.0,9.6,13.5,3.029369e+08,19660.0,69.159021,9072106039,436423300
2013-10-13 22:30:00,2,2013-10-13T00:30:00+02:00,10,37114681000,119543000,2603906116,35230706411,8023402840,27207303571,1007708541,...,11719006,0.0,608873.0,9.6,13.5,3.687257e+08,17004.0,69.212029,9031111381,407924660
2013-10-13 23:00:00,3,2013-10-13T01:00:00+02:00,10,36148675000,136447000,2516689035,34428316838,7982839990,26445476848,966660739,...,11742773,0.0,608873.0,9.6,13.5,4.485727e+08,24299.0,69.318043,8949500729,398239990
2013-10-13 23:30:00,4,2013-10-13T01:30:00+02:00,10,35488831000,164083000,2457628530,33922151279,7953531370,25968619909,929311479,...,11719010,0.0,608873.0,9.5,13.5,5.502433e+08,23308.0,69.450561,8882842849,380571680
2013-10-19 00:00:00,245,2013-10-18T02:00:00+02:00,10,33686594000,129950000,1984788270,32630381752,10483178880,22147202872,1519171839,...,13397012,0.0,607987.0,13.7,12.8,4.875831e+08,15321.0,77.120898,12002350719,338892980
2013-10-19 00:30:00,246,2013-10-18T02:30:00+02:00,10,32214243000,124132000,1878874997,31243739701,10476594310,20767145391,1550189766,...,13170198,0.0,607987.0,13.6,12.8,4.525614e+08,18984.0,77.138169,12026784076,323785680
2013-10-19 01:00:00,247,2013-10-18T03:00:00+02:00,10,31128258000,123587000,1802899886,30215676491,10484512810,19731163681,1543913050,...,13104347,0.0,607987.0,13.6,12.8,4.241059e+08,17968.0,77.259067,12028425860,313428590
2013-10-19 01:30:00,248,2013-10-18T03:30:00+02:00,10,30325643000,135027000,1747919526,29454789181,10537887190,18916901991,1568030620,...,12917777,0.0,607987.0,13.6,12.7,4.160944e+08,17646.0,77.310881,12105917810,306179710
2013-10-19 02:00:00,249,2013-10-18T04:00:00+02:00,10,30077875000,135111000,1731147965,29224557519,10665305180,18559252339,1631709244,...,13051304,0.0,607987.0,13.6,12.7,4.158901e+08,17977.0,77.379965,12297014424,304384000


In [9]:
date.Dataframe(winter_time)

# date.split()

In [10]:
winter_time_workday_df = date.business_days()

winter_time_workday_df.head(20)

,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-10-28 01:00:00,679,2013-10-27T02:00:00+01:00,10,24459666000,3201714000,1532253405,25313829338,7606637680,17707191658,856172708,...,13672053,0.0,608525.0,16.8,11.5,4.868304e+09,2.300200e+04,99.744214,8462810388,216480690
2013-10-28 01:30:00,680,2013-10-27T02:30:00+01:00,10,23375802000,3293724000,1447608994,24251375532,7579418810,16671956722,895502521,...,13602586,0.0,608525.0,16.8,11.5,4.896575e+09,2.190700e+04,99.676005,8474921331,205722940
2013-10-28 02:00:00,681,2013-10-27T03:00:00+01:00,10,22856999000,3441096000,1407491691,23721592891,7566431260,16155161631,931770173,...,13616142,0.0,608525.0,16.8,11.5,4.986335e+09,2.192700e+04,99.641900,8498201433,200179680
2013-10-28 02:30:00,682,2013-10-27T03:30:00+01:00,10,22481941000,3484826000,1378650442,23371139708,7546684550,15824455158,979268804,...,13658500,0.0,608525.0,16.8,11.5,5.022224e+09,2.159600e+04,99.590743,8525953354,196666010
2013-10-28 03:00:00,683,2013-10-27T04:00:00+01:00,10,22292748000,3501652000,1364153040,23180781322,7539947600,15640833722,1023235452,...,13938063,0.0,608525.0,16.7,11.4,5.022595e+09,2.159500e+04,99.522533,8563183052,195489990
2013-10-28 03:30:00,684,2013-10-27T04:30:00+01:00,10,22206676000,3562903000,1357568910,23092682566,7545066880,15547615686,1036813036,...,14442966,0.0,608525.0,16.7,11.4,5.082169e+09,2.258300e+04,99.454324,8581879916,195604310
2013-10-28 04:00:00,685,2013-10-27T05:00:00+01:00,10,22472300000,3587648000,1377910845,23354422246,7563289000,15791133246,1047631558,...,14724222,0.0,608525.0,16.7,11.4,5.127443e+09,2.489800e+04,99.352010,8610920558,198840350
2013-10-28 04:30:00,686,2013-10-27T05:30:00+01:00,10,22752378000,3603827000,1399432969,23626937324,7604854850,16022082474,1058680191,...,14668313,0.0,608525.0,16.7,11.3,5.159787e+09,2.225600e+04,99.283800,8663535041,201157040
2013-10-28 05:00:00,687,2013-10-27T06:00:00+01:00,10,23424540000,3592767000,1451391003,24286480915,7637218760,16649262155,1017920250,...,14681867,0.0,608525.0,16.6,11.3,5.194825e+09,2.192500e+04,99.215591,8655139010,206773320


In [11]:
winter_time_dayoff_df = date.public_holiday()

winter_time_dayoff_df.head(20)


,IDS,Horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),Consommations totale (W),Consommation HTA télérelevée à courbe de charge (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°Celcius),Température normale lissée (°Celcius),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
Horodate_UTC,,,,,,,,,,,,,,,,,,,,,
2013-05-11 00:00:00,1061,2013-11-04T01:00:00+01:00,11,32998581000,3323042000,1934879567,35055530283,8190850810,26864679473,1283633423,...,28209761,0.0,10612737.0,11.7,10.4,5.637112e+09,38510.0,48.885142,9474484233,350019990
2013-05-11 00:30:00,1062,2013-11-04T01:30:00+01:00,11,32495291000,3319788000,1898833635,34596634307,8179182900,26417451407,1242059896,...,28191562,0.0,10612737.0,11.7,10.4,5.634142e+09,36172.0,48.936211,9421242796,331652680
2013-05-11 01:00:00,1063,2013-11-04T02:00:00+01:00,11,32250548000,3391763000,1881446338,34281416225,8215186760,26066229465,1240628063,...,28173363,0.0,10612737.0,11.7,10.4,5.615893e+09,35507.0,48.956639,9455814823,322996280
2013-05-11 01:30:00,1064,2013-11-04T02:30:00+01:00,11,30678651000,3451790000,1771978580,32718475950,8250983570,24467492380,1258580762,...,28080550,0.0,10612737.0,11.7,10.4,5.556135e+09,34179.0,48.997495,9509564332,303862020
2013-05-11 02:00:00,1065,2013-11-04T03:00:00+01:00,11,29518045000,3506311000,1693601803,31555776796,8353441860,23202334936,1314488544,...,27951339,0.0,10612737.0,11.7,10.3,5.525796e+09,36176.0,49.058778,9667930404,287955010
2013-05-11 02:30:00,1066,2013-11-04T03:30:00+01:00,11,28591710000,3508875000,1632538095,30630856154,8494705600,22136150554,1324063419,...,27951339,0.0,10612737.0,11.7,10.3,5.456690e+09,36522.0,49.099634,9818769019,280506970
2013-05-11 03:00:00,1067,2013-11-04T04:00:00+01:00,11,28407171000,3502585000,1620531613,30429930906,8827139000,21602791906,1440522722,...,27951339,0.0,10612737.0,11.7,10.3,5.413937e+09,40183.0,49.120062,10267661722,280265980
2013-05-11 03:30:00,1068,2013-11-04T04:30:00+01:00,11,28579965000,3467853000,1631772374,30595290368,9262992740,21332297628,1447280167,...,28191562,0.0,10612737.0,11.7,10.3,5.386792e+09,31857.0,49.191559,10710272907,280818350
2013-05-11 04:00:00,1069,2013-11-04T05:00:00+01:00,11,30064586000,3369251000,1730251104,32071302046,10162337750,21908964296,1684995031,...,28508219,0.0,10612737.0,11.7,10.2,5.393821e+09,37523.0,49.211987,11847332781,297796720
